In [14]:
import pandas as pd
import numpy as np
import random
from itertools import chain
from IPython.display import clear_output


In [15]:
df = pd.read_csv('example.tsv', sep='\t')
for i in df.index:
    df.loc[i, 'reply'] = ''.join(eval(df.loc[i, 'reply']))
df['label'] = np.zeros(len(df))

df

,question,recognize_result,reply,label
0,是否皆由您本人親自更新網路銀行的帳號與密碼？,NaN,,0.0
1,您好！這裡是玉山銀行總行個金處/OO分行/OO消金中心，敝姓O，員工編號OOOOO，請問是○...,您好這裡是玉山銀行實施宵禁中心敝姓管理員工編號是零九三六是鄰居元先生本人嗎,是,0.0
2,感謝您近期透過本行投保○○人壽○○○，繳費年期為O年/躉繳，依照保險法令的要求，為保障您的權...,感謝您近期透過本行投保南山人壽幸福安康定期貢獻繳費年紀微蹲叫依照保險法的要求回報張您的權益稍...,同意,0.0
3,為維護您的資料安全，這裡簡單跟您核對基本資料，您的身分證字號是，請問後三碼是？,為了維護您的資料安全這邊簡單跟您核對基本資料您的身分證字號是L一二一二二請問後三碼是,八六,0.0
4,請問您的出生年月日是？,請問您的出生年月日是,三好,0.0
...,...,...,...,...
97,請問您投保時，是否皆由○○分行的○○○，在旁邊協助，並由要保人○○○及被保 險人○○○親自簽...,請問您投保時事有南港分行的豬於心從旁協助棒球要保人帳戶先升級被保險人是您本人正副卡先生親自簽...,嗎對好,0.0
98,請問○○分行的○○○是否有向您說明產品內容，並確認符合您的需求？,請問理專是否有說明產品內容請您,,0.0
99,再次與您確認，本保單之規劃您是否已確實瞭解投保目的、保險需求，並經綜合考量財務狀況以及付費能...,請您了解投保目的符合你的保險需求及付款能力並不影響你的日常支出,呃有有有有對對好,0.0
100,請問您本次投保繳交保費的資金來源是否為,請問一下本次保費的資金來源是否為,收入存款我的存款可以可以這麼說,0.0


In [16]:
tmp_df = df[~df['question'].str.contains('出生|身分證|不會|不影響')].copy()
tmp_df = tmp_df[~tmp_df['recognize_result'].isna()]
tmp_df = tmp_df[tmp_df['reply'] != '']
tmp_df = tmp_df.reset_index()
replys = [i for i in tmp_df.reply]

In [30]:
replys_1 = ['是',
'同意',
'嗎我知道好',
'是的好',
'需求請問有好的謝謝',
'有好',
'對謝謝',
'是的謝謝',
'正確是好',
'嘿我是',
'是',
'有說明',
'他說他提供分期分期會比較',
'了解',
'是',
'好',
'好',
'好的了解',
'是好的',
'欸有',
'理解',
'是哪裡玉山銀行嘿對',
'好',
'好的',
'好的',
'好的請問',
'對好',
'本人嗎對對',
'好',
'繳費知道',
'填寫對對',
'需求有有好',
'是否為儲蓄對好',
'了解了解',
'指出好',
'嗎我知道',
'嗎對',
'都知道公司收到',
'對啊',
'嗎可以',
'我知道',
'嗎對好',
'呃有有有有對對好',
'收入存款我的存款可以可以這麼說',
'了解好']
replys_0 = ['小姐',
'先生你好',
'所以',
'保證',
'嗎',
'嗯嗯嗯',
'喔喔嗯',
'什麼意思',
'不好意思',
'阿',
'條款嗎',
'可以詳細說明嗎',
'不確定',
'沒聽說耶',
'是嗎',
'摁',
'是否會儲蓄存款辦理貸款',
'他那個',]
replys = [replys_0, replys_1]

In [33]:
q_series = pd.concat([tmp_df.loc[:, 'question'], tmp_df.loc[:, 'recognize_result']]).reset_index(drop=True)

In [34]:
q_series[:70]

0     您好！這裡是玉山銀行總行個金處/OO分行/OO消金中心，敝姓O，員工編號OOOOO，請問是○...
1     感謝您近期透過本行投保○○人壽○○○，繳費年期為O年/躉繳，依照保險法令的要求，為保障您的權...
2         請問您是否知道本次購買的是○○人壽的保險，不是存款，如果辦理解約將可能只領回部分已繳保費？
3     請問您投保時，是否皆由○○消金中心的○○○，在旁邊協助，並由您本人○○○親自簽名，且被保險人...
4                    請問○○消金中心的○○○是否有向您說明產品內容，並確認符合您的需求？
                            ...                        
65    請問您是否已事先審慎評估自身的財務狀況風險程式的能力願意承擔財務財務槓桿操作方式所面臨的風險...
66                您再次確認上述的投保內容根本貸款並沒有搭上或不但詳細的情形發生請問是否正確
67             請問您知道本子購買安聯人生投資型保險是自行承擔禁止變動的風險並沒有投資方面的保證
68                             請問您知道說繳保費會扣除相關費用後才進入投資帳戶
69                                       請問本是保費的資金來源是否為
Length: 70, dtype: object

In [39]:

res_amount = 200
questions, answers = [], []

q_length = len(tmp_df)

all_q, all_a, labels = [], [], []
for _ in range(res_amount):
    random_var = random.randrange(0, 2)
    q = random.choice(q_series[70:].reset_index(drop=True))
    a = random.choice(replys[random_var])
    all_q.append(q)
    all_a.append(a)
    labels.append(random_var)
    print(q, a, random_var)
    

請問本次投保繳保費的資訊來源是否 嗎對 1
還本終身繳費請問四年依據法律的要求保障接下來我們通話內容將會全程錄音 同意 1
感謝您近期有透過本行投保安聯受豐沛人生變額外受限繳費年紀是躉繳依照保險法要求為了保障您的權益等一下電話服務內容會全程做錄音請問 是好的 1
感謝您近期有透過本行投保安聯受豐沛人生變額外受限繳費年紀是躉繳依照保險法要求為了保障您的權益等一下電話服務內容會全程做錄音請問 喔喔嗯 0
請問地址發行警察先生是否像您說明產品內容辦出院符合你的需求 有好 1
請問您是否有收到本次購買的是中國人壽保險不是存款如果辦理解約像可能只能回部份繳保費 沒聽說耶 0
請問您是否已是先請問評估自身的財務狀況分分風險承受能力定讞萬一財務槓桿操作方式所面臨的風險辦理三解約轉轉投保資料權益損失那處辦理貸款附保單借款去支付本金利息以外還有產品可能發展相關保險局最大可能損失本行人員定位鼓勵或確認有一半貸款保單借款保單解約保單種植定存解約的方式來購買保險請問您是否已了解 收入存款我的存款可以可以這麼說 1
請問您是否知道本子購買的是中國人壽保險不是存款如果辦理解約將可能領回部份繳費 填寫對對 1
請問您在淘寶的時候是否都是我們汐止分行的錢重新在旁邊協助比如說被保險人都是五上漆先生你親自簽名那被保險人健康告知事項與您確認後親自填寫 呃有有有有對對好 1
請問您是否有收到本次購買的是中國人壽保險不是存款如果辦理解約像可能只能回部份繳保費 嗎 0
您好這裡是玉山銀行總行保單不必杏堯員工編號二零二七一好請問是余因為小姐本人嗎 了解好 1
事先審慎評估自身的財務狀況風險承受能力另外承擔財務槓桿操作方式所面臨的風險保單轉投保的權益損失辦理貸款或保單借款去支付本金利息外產品可能發生相關風險其實最大可能損失請問您是否 不確定 0
請問本次投保繳保費的資訊來源是否 都知道公司收到 1
感謝您近期透過本行投保中國人壽的好是零零利率變動型終身保險然後繳保費的年紀為四年那依照保險法的要求到您的權益電話訪問將會全程的錄音請問 填寫對對 1
審慎評估之前的財務狀況風險能力醫院承擔財務槓桿操作方式所面臨的風險保單轉頭告知權益的損失除了辦理貸款或者是保單借款需要支付本金利息以外在產品可能發生的相關風險最大可能的損失請問您是否 需求有有好 1
壽險繳費年紀為六年依據保險法令的要求為保障您的權益接下來我們通話內容會全

In [40]:
train_df = pd.DataFrame({
    'question': all_q,
    'answer': all_a,
    'label': labels,
})

In [41]:
train_df.to_csv('training_data_generated_valid_0119.tsv', sep='\t', index=False)

In [95]:
# 修錯字

In [ ]:
# 加贅字

In [ ]:
# 顧客回答rule

In [ ]:
# 可能沒問 / 問錯rule